In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, cross_val_predict, train_test_split, KFold
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, balanced_accuracy_score, accuracy_score, mean_squared_error
from scipy.stats import pearsonr, ttest_rel
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE, Isomap
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
#import loralib as lora
import random
#import umap
import re
from itertools import product
import os

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Initial Data Processing

In [ ]:
proteomics_data = pd.read_csv('./data/processed/processed_proteomics.csv')

In [ ]:
proteomics_data.index = proteomics_data['UniProt']
proteomics_data = proteomics_data.drop(['Symbol','UniProt'],axis=1).T
proteomics_data.index.name = 'sample_ID'
proteomics_data = proteomics_data.reset_index()
proteomics_data['sample_ID'] = proteomics_data['sample_ID'].astype(int)

In [ ]:
class proteomics_net(nn.Module):
    def __init__(self, input_size, proteomics_hidden_layers, output_size, dropout=0.1):
        super(proteomics_net, self).__init__()
        
        self.proteomics_layers = nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.proteomics_layers(x)
        return x

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)
        

In [ ]:
class GRUNet(nn.Module):
    def __init__(self, input_size_codes, hidden_size, prediction_module_hidden_sizes, num_layers, output_size, dropout=0.1):
        super(GRUNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size_codes, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        self.prediction_module = nn.Sequential(
            nn.Linear(prediction_module_hidden_sizes[0], output_size),
            nn.Sigmoid()
        )



    def forward(self, x, lengths, interpretability=False):
        device = x.device
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out_packed, _ = self.gru(x_packed, h0)
        out, _ = pad_packed_sequence(out_packed, batch_first=True)
        
        out = out[torch.arange(x.size(0)), lengths-1, :]
        out_final = self.prediction_module(out)
        if interpretability == False:
            return out_final
        else:
            return out_final, out

# Prepare the dataset
class PatientDataset(Dataset):
    def __init__(self, data, labels, lengths):
        self.data = data
        self.labels = labels
        self.lengths = lengths

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.lengths[idx]

# Custom collate function for DataLoader
def collate_fn(batch):
    data, labels, lengths = zip(*batch)
    data = pad_sequence(data, batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.float32)
    lengths = torch.tensor(lengths, dtype=torch.long)
    return data, labels, lengths


def normalize_dataset(dataset):
    for i, sequence in enumerate(dataset):
        dataset[i] = (sequence - sequence.mean(dim=0, keepdim=True)) / (sequence.std(dim=0, keepdim=True) + 1e-8)
    return dataset

def impute_missing_values(dataset):
    # Stack all tensors in the dataset along a new dimension, creating a tensor of shape (num_samples, max_seq_length, num_features)
    stacked_data = torch.stack(dataset)

    # Calculate the mean of each feature across all samples and sequences, ignoring NaN values
    feature_means = torch.nanmean(stacked_data, dim=(0, 1))

    # Iterate through the dataset (list of tensors)
    for i, sequence in enumerate(dataset):
        # Create a boolean mask indicating the positions of NaN values in the sequence
        mask = torch.isnan(sequence)

        # Replace NaN values in the sequence with the corresponding feature means
        # 'expand_as' is used to match the dimensions of the mask and the sequence
        dataset[i][mask] = feature_means.expand_as(sequence)[mask]

    return dataset

# Create DataLoaders
def create_dataloaders(patient_data, patient_outcomes, lengths, batch_size=64, normalize=False):
    
    X_train = impute_missing_values(patient_data)
    y_train = patient_outcomes
    
    if normalize:
        X_train = normalize_dataset(X_train)
        
    train_dataset = PatientDataset(X_train, y_train, lengths)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, worker_init_fn=worker_init_fn)
    return train_loader

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)


In [ ]:
# Define the RNN model with GRU units
class joint_model(nn.Module):
    def __init__(self, input_size_codes, hidden_size, prediction_module_hidden_sizes, num_layers, output_size, input_size_proteomics, proteomics_hidden_layers, combined_hidden_layers, dropout=0.1):
        super(joint_model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size_codes, hidden_size, num_layers, batch_first=True, dropout=dropout)
    
        self.prediction_module = nn.Sequential(
            nn.Linear(prediction_module_hidden_sizes[0], output_size)
        )

        self.skip_connect_prot = nn.Linear(input_size_proteomics, output_size)
        
        self.combined_layers = nn.Sequential(
            nn.Linear(input_size_proteomics + hidden_size, output_size)
        )
        
        self.final_combine = nn.Linear(3, 1, bias=False)



    def forward(self, x, x_proteomics, lengths, interpretability=False, better_latent=None, better_ratio=0.5):
        device = x.device
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x_packed = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        out_packed, _ = self.gru(x_packed, h0)
        out, _ = pad_packed_sequence(out_packed, batch_first=True)
        
        out_ehr = out[torch.arange(x.size(0)), lengths-1, :]
        
        if x_proteomics == None:
            return out_ehr

        if better_latent is not None:
            out_ehr = better_ratio * better_latent + (1-better_ratio) * out_ehr
                
        out_combined = torch.cat((out_ehr, x_proteomics), 1)
        
        out_combined = self.combined_layers(out_combined)
        
        pred_proteomics = self.skip_connect_prot(x_proteomics)
        pred_ehr = self.prediction_module(out_ehr)
        
        final_pred = torch.sigmoid(self.final_combine(torch.cat((pred_proteomics, pred_ehr, out_combined), 1)))
#         final_pred = self.final_combine(torch.cat((pred_proteomics, pred_ehr), 1))
        
        if interpretability == False:
            return final_pred
        else:
            return final_pred, (out_ehr, pred_proteomics, pred_ehr, out_combined, final_pred, self.final_combine.weight)
    
    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)

In [ ]:
class DataBuilder(Dataset):
    def __init__(self, x, y, standardizer):
        self.x, self.y, self.standardizer = x, y, standardizer
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return (self.x[index], self.y[index])
    def __len__(self):
        return self.len

In [ ]:
def impute_numpy(mat, column_indices):
    for col_idx in column_indices:
        col_mean = np.nanmean(mat[:, col_idx])  # Compute mean of the column, ignoring nan values
        nan_idx = np.isnan(mat[:, col_idx])  # Get boolean mask of nan values in the column
        mat[nan_idx, col_idx] = col_mean  # Replace nan values with the column mean
    return mat

In [ ]:
def run_experiment(EHR_codes, proteomics, patient_indices, outcomes, lengths, experiment_name, lr, lr_decay,
                   bs, train_indices=None, val_indices=None, test_indices=None, feature_types='EHR', model_path='', fine_tune=False, seed=42, num_layers=2,hidden_dim=400,
                   dropout=0.4, return_preds=False, return_interpretability=False, return_grads=False,
                   protein_weights=None, blend=None, hyperparam_tuning=False):
    """
    EHR_codes: pre-processed data for codes of shape (num_patients, max_length, embedding_dim)v
    EHR_vitals: pre-processed data for codes of shape (num_patients, max_length, embedding_dim)
    proteomics: dataframe with proteomics data
    patient_indices: dataframe with sample IDs and row numbers in pre-processed matrices
    outcomes: array with DOS
    lengths: array with lengths (i.e. number of visits) to help with padding
    experiment_name: string for file name for models
    lr: float for learning rate
    lr_decay: float for learning rate decay
    bs: int for batch size
    feature_types: string either 'EHR', 'metab', 'both'
    model_path: string for file path to model if loading a pre-trained model
    fine_tune: boolean for whether or not EHR weight should be learned, can only be true if model != ''
    seed: int, random_seed for train/test/val split and seeding model 
    ...
    blend: if True, blend noise to PT model; if file path, blend real weights to NPT model (not implemented yet)
    """
    set_seed(seed)
    prediction_module_hidden_sizes = [hidden_dim,hidden_dim//2, hidden_dim//4, hidden_dim//8]
    
    assert feature_types in ['EHR','metab','both']   
    if feature_types == 'metab': assert model_path == ''
    if (model_path != '') & (feature_types == 'both'): assert fine_tune==True
    if hyperparam_tuning == False: assert train_indices == None

    
    if hyperparam_tuning == False:
        maternal_IDs = patient_indices['sample_ID']

        train_ratio = 0.70
        test_ratio = 0.15
        val_ratio = 0.15

        # First, split the unique_ids into train and temp (test + validation) sets
        train_ids, temp_ids = train_test_split(maternal_IDs, test_size=(test_ratio + val_ratio),random_state=seed, stratify=outcomes)
        
        patient_indices_temp = patient_indices.copy(deep=True)

        patient_indices_temp.index = patient_indices_temp['sample_ID']
        temp_indices = patient_indices_temp.loc[temp_ids,'array_index'].values

        # Next, split the temp_ids into test and validation sets
        test_ids, val_ids = train_test_split(temp_ids, test_size=(val_ratio / (test_ratio + val_ratio)), random_state=seed, stratify=outcomes[temp_indices])
        proteomics = proteomics.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices[patient_indices['sample_ID'].isin(train_ids)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['sample_ID'].isin(test_ids)]['array_index'].values
        val_indices = patient_indices[patient_indices['sample_ID'].isin(val_ids)]['array_index'].values
        counter = seed+100
        while (len(train_indices) % bs == 1) | (len(test_indices) % bs == 1) | (len(val_indices) % bs == 1):
            counter += 1
            if counter > 150:
                print('stuck!')
                break
            train_ids, temp_ids = train_test_split(maternal_IDs, test_size=(test_ratio + val_ratio), random_state=counter)
            test_ids, val_ids = train_test_split(temp_ids, test_size=(val_ratio / (test_ratio + val_ratio)), random_state=counter)
            train_indices = patient_indices[patient_indices['sample_ID'].isin(train_ids)]['array_index'].values
            np.random.shuffle(train_indices)
            test_indices = patient_indices[patient_indices['sample_ID'].isin(test_ids)]['array_index'].values
            val_indices = patient_indices[patient_indices['sample_ID'].isin(val_ids)]['array_index'].values

    train_EHR_codes = EHR_codes[train_indices,:,:]
    if feature_types != 'EHR':
        train_proteomics = proteomics[train_indices,:]
        scaler = StandardScaler()
        train_proteomics = scaler.fit_transform(train_proteomics)
    train_outcomes = outcomes[train_indices]
    train_lengths = lengths[train_indices]
    
    test_EHR_codes = EHR_codes[test_indices, :, :]
    if feature_types != 'EHR':
        test_proteomics = proteomics[test_indices, :]
        scaler = StandardScaler()
        test_proteomics = scaler.fit_transform(test_proteomics)
    test_outcomes = outcomes[test_indices]
    test_lengths = lengths[test_indices]

    val_EHR_codes = EHR_codes[val_indices, :, :]
    if feature_types != 'EHR':
        val_proteomics = proteomics[val_indices, :]
        scaler = StandardScaler()
        val_proteomics = scaler.fit_transform(val_proteomics)
    val_outcomes = outcomes[val_indices]
    val_lengths = lengths[val_indices]
        
    all_EHR_codes = EHR_codes
    scaler = StandardScaler()
    all_outcomes = outcomes
    all_lengths = lengths

    train_EHR_codes = [torch.tensor(data).float() for data in train_EHR_codes]  
    train_EHR_codes = [torch.nan_to_num(x) for x in train_EHR_codes]
    if feature_types != 'EHR':
        train_proteomics = torch.tensor(train_proteomics).float()
        train_proteomics = torch.nan_to_num(train_proteomics)
    train_outcomes = torch.tensor(train_outcomes).float()
    
        # Test sets
    test_EHR_codes = [torch.tensor(data).float() for data in test_EHR_codes]
    test_EHR_codes = [torch.nan_to_num(x) for x in test_EHR_codes]
    if feature_types != 'EHR':
        test_proteomics = torch.tensor(test_proteomics).float()
        test_proteomics = torch.nan_to_num(test_proteomics)
    test_outcomes = torch.tensor(test_outcomes).float()

    # Validation sets
    val_EHR_codes = [torch.tensor(data).float() for data in val_EHR_codes]
    val_EHR_codes = [torch.nan_to_num(x) for x in val_EHR_codes]
    if feature_types != 'EHR':
        val_proteomics = torch.tensor(val_proteomics).float()
        val_proteomics = torch.nan_to_num(val_proteomics)
    val_outcomes = torch.tensor(val_outcomes).float()
    
    all_EHR_codes = [torch.tensor(data).float() for data in all_EHR_codes]
    all_EHR_codes = [torch.nan_to_num(x) for x in all_EHR_codes]
    all_outcomes = torch.tensor(all_outcomes).float()

    if feature_types != 'EHR':
        data_set_train = DataBuilder(train_proteomics, train_outcomes, scaler)
        train_loader_proteomics = DataLoader(dataset=data_set_train,batch_size=bs, worker_init_fn=worker_init_fn)
    train_loader_codes = create_dataloaders(train_EHR_codes, train_outcomes, train_lengths, bs)

    if feature_types != 'EHR':
        data_set_test = DataBuilder(test_proteomics, test_outcomes, scaler)
        test_loader_proteomics = DataLoader(dataset=data_set_test,batch_size=bs, worker_init_fn=worker_init_fn)
    test_loader_codes = create_dataloaders(test_EHR_codes, test_outcomes, test_lengths, bs)

    if feature_types != 'EHR':
        data_set_val = DataBuilder(val_proteomics, val_outcomes, scaler)
        val_loader_proteomics = DataLoader(dataset=data_set_val,batch_size=100*bs, worker_init_fn=worker_init_fn)
    val_loader_codes = create_dataloaders(val_EHR_codes, val_outcomes, val_lengths, bs)
            
    all_loader_codes = create_dataloaders(all_EHR_codes, all_outcomes, all_lengths, 1000)   
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    if model_path == '':
        if feature_types == 'EHR':
            model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout).to(device)
        elif feature_types == 'metab':
            model = proteomics_net(proteomics.shape[1], [1024, 512, 256, 128], 1, dropout).to(device)
        elif feature_types == 'both':
            model = joint_model(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, proteomics.shape[1], [1024, 512, 256, 128], [64, 32, 16, 8], dropout).to(device)
            
    else:
        if feature_types == 'EHR':
            model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout)
            model_state_dict = torch.load(model_path)
            model.load_state_dict(model_state_dict)
            model.to(device)
            if fine_tune == False:
                model.eval()
                criterion = nn.BCELoss()
                val_predictions = []
                val_true_labels = []
                running_loss_val, num_samples_val = 0, 0
                with torch.no_grad():
                    for (inputs_codes, labels_codes, lengths_codes) in (val_loader_codes):
                            inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                            outputs = model(inputs_codes, lengths_codes)

                            loss = criterion(outputs.squeeze(), labels)
                            running_loss_val += (loss.item()*lengths_codes.shape[0])
                            num_samples_val += lengths_codes.shape[0]
                            val_predictions.extend(outputs.squeeze().tolist())
                            val_true_labels.extend(labels.tolist())
                    
                val_loss = running_loss_val / (num_samples_val)
                pearson_corr = roc_auc_score(val_true_labels, val_predictions)

                print(f'Total Loss: {val_loss:.4f}, Pearson R: {pearson_corr:.4f}')
                if return_preds == True:
                    return pearson_corr, val_loss, None, val_true_labels, val_predictions, val_indices
                else:
                    return pearson_corr, val_loss, None
            elif fine_tune == True:
                model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout)
                model_state_dict = torch.load(model_path)
                model.load_state_dict(model_state_dict, strict=False)
                model.to(device)
                for name, param in model.named_parameters():
                    if ('gru' in name):
                        param.requires_grad = False

        elif feature_types == 'both':
            model = joint_model(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, proteomics.shape[1], [1024, 512, 256, 128], [64, 32, 16, 8], dropout).to(device)
            model_state_dict = torch.load(model_path)
            gru_weights = {}
            for k,v in zip(model_state_dict.keys(), model_state_dict.values()):
                if ('gru' in k) | ('pred' in k):
                    gru_weights[k] = v
            model.load_state_dict(gru_weights, strict=False)
            model.to(device)
            
            for name, param in model.named_parameters():
                if ('gru' in name):
                    param.requires_grad = False
    
    if protein_weights != None:
        model_state_dict = torch.load(protein_weights)
        protein_weight_dict = {}
        for k,v in zip(model_state_dict.keys(), model_state_dict.values()):
            if ('skip_connect' in k) | ('combined_layers' in k):
                protein_weight_dict[k] = v
        model.load_state_dict(protein_weight_dict, strict=False)
                
   
    if blend != None:
        PT_EHR_model = GRUNet(EHR_codes.shape[2], hidden_dim, prediction_module_hidden_sizes, num_layers, 1, dropout)
        model_state_dict = torch.load(blend)
        PT_EHR_model.load_state_dict(model_state_dict)
        PT_EHR_model.to(device)
        PT_EHR_model.eval()

        
    epoch_arr = []
    num_samples_in_batch = []
    gradient_arr = []
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=lr_decay)
    num_epochs = 200
    train_losses = []
    test_losses = []
    val_losses = []
    best_loss = np.inf
    for epoch in tqdm(range(num_epochs)):
        model.train()
        if hyperparam_tuning == False:
            torch.cuda.synchronize()  # Wait for all CUDA kernels to finish
            torch.save(model.state_dict(), './models/predictive_models/{}_epoch{}.pth'.format(experiment_name, epoch))
        running_loss_train, num_train_samples = 0, 0
        if feature_types == 'EHR':
            for (inputs_codes, labels_codes, lengths_codes)in train_loader_codes:
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)

                optimizer.zero_grad()
                
                if blend != None:
                    with torch.no_grad():
                        outputs = PT_EHR_model(inputs_codes, lengths_codes, True)
                        optimal_latent = outputs[1]
                    outputs = model(inputs_codes, lengths_codes, better_latent=optimal_latent, better_ratio=0.5)
                
                else:
                    outputs = model(inputs_codes, lengths_codes)

                loss = criterion(outputs.squeeze(), labels)

                loss.backward()
                optimizer.step()
                running_loss_train += (loss.item()*lengths_codes.shape[0])
                num_train_samples += lengths_codes.shape[0]
        else:
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(train_loader_codes, train_loader_proteomics)):
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                if feature_types != 'EHR':
                    inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                optimizer.zero_grad()
                if feature_types == 'metab':
                    outputs = model(inputs_proteomics)
                elif feature_types == 'EHR':
                    outputs = model(inputs_codes, lengths_codes)
                elif feature_types == 'both':
                    outputs, interpretability_outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)

                loss = criterion(outputs.squeeze(), labels)
                loss.backward()
                optimizer.step()
                if return_grads:
                    epoch_arr.append(epoch)
                    num_samples_in_batch.append(outputs.shape[0])
                    gradient_arr.append(model.skip_connect_prot.weight.grad.cpu().numpy()[0])
                running_loss_train += (loss.item()*lengths_codes.shape[0])
                num_train_samples += lengths_codes.shape[0]

        train_loss = running_loss_train / num_train_samples
        train_losses.append(train_loss)

        model.eval()
        running_loss_test, num_samples_test = 0, 0
        predictions = []
        true_labels = []
        if feature_types == 'EHR':
            with torch.no_grad():
                for (inputs_codes, labels_codes, lengths_codes)in test_loader_codes:
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    outputs = model(inputs_codes, lengths_codes)
                    outputs = outputs.squeeze(1)
                    loss = criterion(outputs, labels)
                    running_loss_test += (loss.item()*lengths_codes.shape[0])
                    num_samples_test += lengths_codes.shape[0]
                    predictions.extend(outputs.tolist())
                    true_labels.extend(labels.tolist())
        else:
            with torch.no_grad():
                for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(test_loader_codes, test_loader_proteomics)):
                    if feature_types != 'metab':
                        inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                    if feature_types != 'EHR':
                        inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                    if feature_types == 'metab':
                        outputs = model(inputs_proteomics)
                    elif feature_types == 'EHR':
                        outputs = model(inputs_codes, lengths_codes)
                    elif feature_types == 'both':
                        outputs = model(inputs_codes, inputs_proteomics, lengths_codes)
                    
                    outputs = outputs.squeeze(1)
                    loss = criterion(outputs, labels)
                    
                    running_loss_test += (loss.item()*lengths_codes.shape[0])
                    num_samples_test += lengths_codes.shape[0]
                    predictions.extend(outputs.tolist())
                    true_labels.extend(labels.tolist())
        test_loss = running_loss_test / (num_samples_test)
        test_losses.append(test_loss)
        
        patience = 5
        pearson_corr = roc_auc_score(true_labels, predictions)

        print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Pearson R: {pearson_corr:.4f}')
        
        if test_loss < best_loss:
            best_loss = test_loss
            counter = 0
            torch.cuda.synchronize()  # Wait for all CUDA kernels to finish
            torch.save(model.state_dict(), './models/predictive_models/{}.pth'.format(experiment_name))
        else:
            counter += 1
            if counter >= patience:
                break

    
    model.load_state_dict(torch.load('./models/predictive_models/{}.pth'.format(experiment_name)))
    running_loss_val, num_samples_val = 0, 0
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        model.eval()
        if feature_types == 'EHR':
            for (inputs_codes, labels_codes, lengths_codes) in val_loader_codes:
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                if feature_types != 'EHR':
                    inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                if feature_types == 'metab':
                    outputs = model(inputs_proteomics)
                elif feature_types == 'EHR':
                    outputs = model(inputs_codes, lengths_codes)
                elif feature_types == 'both':
                    outputs, interpretability_outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)
                    
                loss = criterion(outputs.squeeze(), labels)
                running_loss_val += (loss.item()*lengths_codes.shape[0])
                num_samples_val += lengths_codes.shape[0]
                val_predictions.extend(outputs.squeeze().tolist())
                val_true_labels.extend(labels.tolist())
        else:
            for (inputs_codes, labels_codes, lengths_codes), (inputs_proteomics, labels_proteomics) in (zip(val_loader_codes, val_loader_proteomics)):
                if feature_types != 'metab':
                    inputs_codes, labels = inputs_codes.to(device), labels_codes.to(device)
                if feature_types != 'EHR':
                    inputs_proteomics, labels = inputs_proteomics.to(device), labels_proteomics.to(device)

                if feature_types == 'metab':
                    outputs = model(inputs_proteomics)
                elif feature_types == 'EHR':
                    outputs = model(inputs_codes, lengths_codes)
                elif feature_types == 'both':
                    outputs, interpretability_outputs = model(inputs_codes, inputs_proteomics, lengths_codes, interpretability=True)

                loss = criterion(outputs.squeeze(), labels)
                running_loss_val += (loss.item()*lengths_codes.shape[0])
                num_samples_val += lengths_codes.shape[0]
                val_predictions.extend(outputs.squeeze().tolist())
                val_true_labels.extend(labels.tolist())
    val_loss = running_loss_val / (num_samples_val)
    val_losses.append(val_loss)
    pearson_corr = roc_auc_score(val_true_labels, val_predictions)

    print(f'Epoch: {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Pearson R: {pearson_corr:.4f}')
    
    if hyperparam_tuning == True:
        os.remove('./models/predictive_models/{}.pth'.format(experiment_name))
    if return_preds == True:
        if return_interpretability == False:
            if return_grads:
                df = pd.DataFrame([epoch_arr, num_samples_in_batch, gradient_arr]).T
                df.columns = ['epoch', 'num_samples_in_batch','gradient']
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices, df, train_losses, test_losses, val_losses, train_indices, test_indices
            else:
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices
        else:
            if return_grads:
                df = pd.DataFrame([epoch_arr, num_samples_in_batch, gradient_arr]).T
                df.columns = ['epoch', 'num_samples_in_batch','gradient']
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices, interpretability_outputs, None, None, df, train_losses, test_losses, val_losses, train_indices, test_indices
            else:
                return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices, interpretability_outputs, None, None
    else:
        return pearson_corr, val_loss, None



In [ ]:
RNN_data_codes_omics = np.load('./data/processed/RNN_data_omics_cohort_disease_modeling.npy')
RNN_data_outcomes_omics = np.load('./data/processed/RNN_data_outcomes_omics_cohort_disease_status.npy')
RNN_data_lengths_omics = np.load('./data/processed/RNN_data_lengths_omics_cohort_disease_status.npy')
patient_indices_omics = pd.read_csv('./data/processed/sampleID_indices_omics_cohort_disease_modeling.csv')
patient_indices_omics.columns = ['sample_ID','array_index']
RNN_data_codes_omics.shape, RNN_data_outcomes_omics.shape, RNN_data_lengths_omics.shape, patient_indices_omics.shape

In [ ]:
(RNN_data_outcomes_omics == 1).sum()

In [ ]:
RNN_data_codes = np.load('./data/processed/RNN_data_omics_cohort_disease_modeling_PT_word2vec_model.npy')
RNN_data_outcomes = np.load('./data/processed/RNN_data_outcomes_omics_cohort_disease_status_PT_word2vec_model.npy')
RNN_data_lengths = np.load('./data/processed/RNN_data_lengths_omics_cohort_disease_status_PT_word2vec_model.npy')
patient_indices = pd.read_csv('./data/processed/sampleID_indices_omics_cohort_disease_modeling_PT_word2vec_model.csv')
patient_indices.columns = ['sample_ID','array_index']
RNN_data_codes.shape, RNN_data_outcomes.shape, RNN_data_lengths.shape, patient_indices.shape

In [ ]:
RNN_data_codes_PTMODEL = np.load('./data/processed/RNN_data_full_EHR_cohort_disease_classification.npy')
RNN_data_outcomes_PTMODEL = np.load('./data/processed/RNN_data_outcomes_full_EHR_cohort_disease_classification.npy')
RNN_data_lengths_PTMODEL = np.load('./data/processed/RNN_data_lengths_full_EHR_cohort_disease_classification.npy')
patient_indices_PTMODEL = pd.read_csv('./data/processed/sampleID_indices_full_cohort_disease_classification.csv')
patient_indices_PTMODEL.columns = ['sample_ID','array_index']
RNN_data_codes_PTMODEL.shape, RNN_data_outcomes_PTMODEL.shape, RNN_data_lengths_PTMODEL.shape, patient_indices_PTMODEL.shape

In [ ]:
overall_best_params = {}

In [ ]:
outcome_list = ['DR']

In [ ]:
# Hyperparameter grid
grid_search = {
    'batch_size': [512],
    'lr': [1e-2, 1e-3, 1e-4],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
    'layers': [2, 4],
    'hidden_dim': [400, 800]
}

# Generate all combinations of hyperparameters
all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]


num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i] = {'PT': {}}

    maternal_IDs = patient_indices_PTMODEL['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in tqdm(kf.split(maternal_IDs)):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        

        train_indices = patient_indices_PTMODEL[patient_indices_PTMODEL['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_PTMODEL[patient_indices_PTMODEL['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_PTMODEL[patient_indices_PTMODEL['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            layers = param_set['layers']
            hidden_dim = param_set['hidden_dim']
            
            model_name = 'PT_MODEL_{}_{}_{}_{}_{}'.format(layers,lr,lr_decay,dropout, hidden_dim)
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_PTMODEL, proteomics_data,
            patient_indices_PTMODEL, RNN_data_outcomes_PTMODEL, RNN_data_lengths_PTMODEL, model_name, 
            lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                                                       val_indices=val_indices,feature_types='EHR', model_path='', fine_tune=False, seed=42, 
            hidden_dim=hidden_dim, num_layers=layers, dropout=dropout, hyperparam_tuning=True)
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('PT')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
model_name = 'PT_MODEL_{}_{}_{}_{}_{}'.format(num_layers,lr,lr_decay,dropout, hidden_dim)
overall_best_params['DR']['PT'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs), 'model_name': model_name}
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
overall_best_params = pickle.load(open('./models/hyperparameters/best_hyperparams.pkl','rb'))

In [ ]:
best_num_layers = overall_best_params['DR']['PT']['num_layers']
best_dropout = overall_best_params['DR']['PT']['dropout']
best_model_name = overall_best_params['DR']['PT']['model_name']
best_hidden_dim = overall_best_params['DR']['PT']['hidden_dim']
best_num_layers, best_dropout, best_hidden_dim, best_model_name

In [ ]:
%%time
val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_PTMODEL, proteomics_data,
    patient_indices_PTMODEL, RNN_data_outcomes_PTMODEL, RNN_data_lengths_PTMODEL, best_model_name, 
    overall_best_params['DR']['PT']['lr'], overall_best_params['DR']['PT']['lr_decay'],
       overall_best_params['DR']['PT']['batch_size'], feature_types='EHR', model_path='', fine_tune=False, seed=42, 
    hidden_dim=best_hidden_dim, num_layers=best_num_layers, dropout=best_dropout, hyperparam_tuning=False)

In [ ]:
grid_search = {
    'batch_size': [16],
    'lr': [1e-1, 1e-2, 1e-3, 1e-4],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4],
    'layers': [2, 4],
    'hidden_dim': [400, 800]
}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
np.random.seed(3)

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp1'] = {}

    maternal_IDs = patient_indices_omics['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in tqdm(kf.split(maternal_IDs)):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics_data.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            layers = param_set['layers']
            hidden_dim = param_set['hidden_dim']
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_omics, input_proteomics,
                                                       patient_indices_omics, RNN_data_outcomes_omics,
                                                       RNN_data_lengths_omics, 'EHR_omics_only',
                                                       lr, lr_decay, bs, 
                                                       train_indices=train_indices, test_indices=test_indices,
                                                       val_indices=val_indices, feature_types='EHR', model_path='',
                                                       fine_tune=False, seed=42, hidden_dim=hidden_dim,
                                                       num_layers=layers, dropout=dropout, hyperparam_tuning=True)

            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))

print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['DR']['exp1'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
best_num_layers_omics = overall_best_params['DR']['exp1']['num_layers']
best_dropout_omics = overall_best_params['DR']['exp1']['dropout']
best_hidden_dim_omics = overall_best_params['DR']['exp1']['hidden_dim']
best_num_layers_omics, best_dropout_omics, best_hidden_dim_omics

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
np.random.seed(3)

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp2'] = {}

    maternal_IDs = patient_indices_omics['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in tqdm(kf.split(maternal_IDs)):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics_data.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

        train_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout_omics
            lr_decay = param_set['lr_decay']
            layers = best_num_layers_omics
            hidden_dim = best_hidden_dim_omics
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_omics, input_proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'proteomics_omics_only', 
                lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices, val_indices=val_indices,
                feature_types='metab', model_path='', fine_tune=False, seed=42, hidden_dim=hidden_dim,
                num_layers=layers, dropout=dropout, hyperparam_tuning=True)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 2')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['DR']['exp2'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
np.random.seed(3)

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp3'] = {}

    maternal_IDs = patient_indices_omics['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in tqdm(kf.split(maternal_IDs)):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics_data.merge(patient_indices_omics[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

        train_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices_omics[patient_indices_omics['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            layers = best_num_layers_omics
            hidden_dim = best_hidden_dim_omics
            print(param_set)
            
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes_omics, input_proteomics,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'both_omics_only', 
                lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices, feature_types='both', model_path='', fine_tune=False, seed=42,
                hidden_dim=hidden_dim, num_layers=layers, dropout=dropout, hyperparam_tuning=True)

            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 3')
        print('outcome {}'.format(i))


In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['DR']['exp3'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
np.random.seed(3)
for i in tqdm(outcome_list):
    overall_best_params[i]['exp4'] = {}
    results_dict = {}

    # Iterate through the hyperparameter combinations
    bs = 1000
    lr = 0.1
    dropout = best_dropout
    lr_decay = 0.1
    layers = best_num_layers
    hidden_dim = best_hidden_dim
    val_r, val_loss, val_rmse = run_experiment(RNN_data_codes, proteomics_data,
            patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_OOL_PT', 
            lr, lr_decay, bs, feature_types='EHR', model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                                               fine_tune=False, seed=42,
                                               hidden_dim=hidden_dim,
                                              num_layers=layers, dropout=dropout, hyperparam_tuning=False)

    results_dict[val_loss] = {'num_layers': layers,'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                            'hidden_dim': hidden_dim, 'batch_size': bs}

    print('experiment 4')
    print('outcome {}'.format(i))
    overall_best_params[i]['exp4'] = results_dict[min(results_dict.keys())]
    print(results_dict[min(results_dict.keys())])

# Save the best hyperparameters to a pickle file
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
    pickle.dump(overall_best_params, f)

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
np.random.seed(3)

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp5'] = {}

    maternal_IDs = patient_indices['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in tqdm(kf.split(maternal_IDs)):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)

        input_proteomics = proteomics_data.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values

        train_indices = patient_indices[patient_indices['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices[patient_indices['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout
            lr_decay = param_set['lr_decay']
            layers = best_num_layers
            hidden_dim = best_hidden_dim
            
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes, input_proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_OOL_PT', 
                lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices, feature_types='EHR',
                model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                fine_tune=True, seed=42, hidden_dim=hidden_dim, num_layers=layers, dropout=dropout,
                                                       hyperparam_tuning=True)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 5')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)
overall_best_params['DR']['exp5'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
grid_search = {'batch_size': [16],
              'lr': [1e-1, 1e-2, 1e-3, 1e-4],
              'lr_decay': [1e-1, 1e-2, 1e-3, 1e-4]}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
np.random.seed(3)

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp6'] = {}

    maternal_IDs = patient_indices['sample_ID'].unique()
    
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in tqdm(kf.split(maternal_IDs)):
        split_num += 1
        results_dict = {}
        train_IDs = maternal_IDs[train_index]
        test_IDs = maternal_IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        
        input_proteomics = proteomics_data.merge(patient_indices[['sample_ID','array_index']], how='left', left_on='sample_ID', right_on='sample_ID').drop(['sample_ID'],axis=1).sort_values('array_index').drop('array_index',axis=1).values


        train_indices = patient_indices[patient_indices['sample_ID'].isin(train_IDs)]['array_index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['sample_ID'].isin(test_IDs)]['array_index'].values
        val_indices = patient_indices[patient_indices['sample_ID'].isin(val_IDs)]['array_index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout
            lr_decay = param_set['lr_decay']
            layers = best_num_layers
            hidden_dim = best_hidden_dim
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(RNN_data_codes, input_proteomics,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'both_PT', 
                lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices,feature_types='both',
                model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                fine_tune=True, seed=42, hidden_dim=hidden_dim, num_layers=layers, dropout=dropout,
                                                        hyperparam_tuning=True)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 6')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()

print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['DR']['exp6'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./models/hyperparameters/best_hyperparams.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
overall_best_params = pickle.load(open('./models/hyperparameters/best_hyperparams.pkl','rb'))

In [ ]:
best_num_layers = overall_best_params['DR']['PT']['num_layers']
best_dropout = overall_best_params['DR']['PT']['dropout']
best_model_name = overall_best_params['DR']['PT']['model_name']
best_hidden_dim = overall_best_params['DR']['PT']['hidden_dim']
best_num_layers, best_dropout, best_hidden_dim, best_model_name

In [ ]:
best_num_layers_OOL = overall_best_params['DR']['exp1']['num_layers']
best_dropout_OOL = overall_best_params['DR']['exp1']['dropout']
best_hidden_dim_OOL = overall_best_params['DR']['exp1']['hidden_dim']
best_num_layers_OOL, best_dropout_OOL, best_hidden_dim_OOL

In [ ]:
# experiment 1 = CB model EHR features
#experiment 2 = CB model metab features
#experiment 3 = CB model all features
#experiment 4 = only pretrained model
#experiment 5 = fine tune pretrained model 
#experiment 6 = add metabs, fine tune pretrained model

num_iterations = 25

results = {}
for i in tqdm(outcome_list):
    results[i] = {'exp1':[],'exp2':[],'exp3':[],'exp4':[],'exp5':[],'exp6':[]}
    for j in tqdm(range(num_iterations)):
        print('experiment 1')
        val_auc = run_experiment(RNN_data_codes_omics, proteomics_data,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'EHR_omics_only_{}'.format(j), 
                overall_best_params[i]['exp1']['lr'], overall_best_params[i]['exp1']['lr_decay'],
                overall_best_params[i]['exp1']['batch_size'], feature_types='EHR', model_path='', fine_tune=False, seed=j,
                                hidden_dim=overall_best_params[i]['exp1']['hidden_dim'],
                                 num_layers=overall_best_params[i]['exp1']['num_layers'],
                                 dropout=overall_best_params[i]['exp1']['dropout'], return_preds=True)
        results[i]['exp1'].append(val_auc)
        
        print('experiment 2')
        val_auc = run_experiment(RNN_data_codes_omics, proteomics_data,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'proteomics_omics_only_{}'.format(j), 
                overall_best_params[i]['exp2']['lr'], overall_best_params[i]['exp2']['lr_decay'],
                overall_best_params[i]['exp2']['batch_size'], feature_types='metab', model_path='', fine_tune=False, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers, dropout=best_dropout,
                                 return_preds=True)
        results[i]['exp2'].append(val_auc)
        
        print('experiment 3')
        val_auc = run_experiment(RNN_data_codes_omics, proteomics_data,
                patient_indices_omics, RNN_data_outcomes_omics, RNN_data_lengths_omics, 'both_omics_only_{}'.format(j), 
                overall_best_params[i]['exp3']['lr'], overall_best_params[i]['exp3']['lr_decay'],
                overall_best_params[i]['exp3']['batch_size'], feature_types='both', model_path='', fine_tune=False, seed=j,
                                hidden_dim=overall_best_params[i]['exp3']['hidden_dim'],
                                 num_layers=overall_best_params[i]['exp3']['num_layers'],
                                 dropout=overall_best_params[i]['exp3']['dropout'],
                                 return_preds=True, return_interpretability=True, return_grads=True)
        results[i]['exp3'].append(val_auc)
        
        print('experiment 4')
        val_auc = run_experiment(RNN_data_codes, proteomics_data,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_PT_{}'.format(j), 
                overall_best_params[i]['exp4']['lr'], overall_best_params[i]['exp4']['lr_decay'],
                overall_best_params[i]['exp4']['batch_size'], feature_types='EHR',
                                 model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                                 fine_tune=False, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout, return_preds=True)
        results[i]['exp4'].append(val_auc)

        print('experiment 5')
        val_auc = run_experiment(RNN_data_codes, proteomics_data,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'EHR_PT_FT_{}'.format(j), 
                overall_best_params[i]['exp5']['lr'], overall_best_params[i]['exp5']['lr_decay'],
                overall_best_params[i]['exp5']['batch_size'], feature_types='EHR',
                                 model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                                 fine_tune=True, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout, return_preds=True)
        results[i]['exp5'].append(val_auc)

        print('experiment 6')
        val_auc = run_experiment(RNN_data_codes, proteomics_data,
                patient_indices, RNN_data_outcomes, RNN_data_lengths, 'both_PT_FT_{}'.format(j), 
                overall_best_params[i]['exp6']['lr'], overall_best_params[i]['exp6']['lr_decay'],
                overall_best_params[i]['exp6']['batch_size'], feature_types='both',
                                model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                                 fine_tune=True, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout,
                               return_preds=True, return_interpretability=True, return_grads=True)
        results[i]['exp6'].append(val_auc)
    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score

# Function to convert tensors to numpy arrays
def tensor_to_numpy(x):
    if hasattr(x, 'numpy'):  # Check if it's a tensor with numpy method
        return x.numpy().astype(float)
    elif 'tensor' in str(type(x)).lower():  # Check if it's a tensor by name
        try:
            return float(str(x).split('(')[1].split(')')[0])
        except:
            return float(x)
    elif isinstance(x, (int, float, np.number)):
        return float(x)
    else:
        try:
            return float(x)
        except:
            raise ValueError(f"Cannot convert {type(x)} to float: {x}")

# Bootstrap confidence intervals for AUROC
def bootstrap_auroc_ci(y_true, y_pred, n_bootstraps=1000, ci=0.95):
    # Convert to numpy arrays from any potential tensor objects
    y_true_np = np.array([tensor_to_numpy(y) for y in y_true])
    y_pred_np = np.array([tensor_to_numpy(y) for y in y_pred])
    
    bootstrapped_scores = []
    rng = np.random.RandomState(42)
    
    for i in range(n_bootstraps):
        # Bootstrap by sampling with replacement
        indices = rng.randint(0, len(y_true_np), len(y_true_np))
        
        # Check if the bootstrapped sample has both classes
        if len(np.unique(y_true_np[indices])) < 2:
            continue
        
        try:
            score = roc_auc_score(y_true_np[indices], y_pred_np[indices])
            bootstrapped_scores.append(score)
        except Exception as e:
            continue
    
    if len(bootstrapped_scores) == 0:
        return np.nan, np.nan, np.nan
    
    # Calculate point estimate
    auroc = roc_auc_score(y_true_np, y_pred_np)
    
    # Sort scores and get confidence interval bounds
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    
    # Compute confidence interval
    alpha = (1.0 - ci) / 2.0
    lower_bound = sorted_scores[int(alpha * len(sorted_scores))]
    upper_bound = sorted_scores[int((1 - alpha) * len(sorted_scores))]
    
    return auroc, lower_bound, upper_bound

# Bootstrap confidence intervals for AUPRC
def bootstrap_auprc_ci(y_true, y_pred, n_bootstraps=1000, ci=0.95):
    # Convert to numpy arrays from any potential tensor objects
    y_true_np = np.array([tensor_to_numpy(y) for y in y_true])
    y_pred_np = np.array([tensor_to_numpy(y) for y in y_pred])
    
    bootstrapped_scores = []
    rng = np.random.RandomState(42)
    
    for i in range(n_bootstraps):
        # Bootstrap by sampling with replacement
        indices = rng.randint(0, len(y_true_np), len(y_true_np))
        
        # Check if the bootstrapped sample has both classes
        if len(np.unique(y_true_np[indices])) < 2:
            continue
        
        try:
            score = average_precision_score(y_true_np[indices], y_pred_np[indices])
            bootstrapped_scores.append(score)
        except Exception as e:
            continue
    
    if len(bootstrapped_scores) == 0:
        return np.nan, np.nan, np.nan
    
    # Calculate point estimate
    auprc = average_precision_score(y_true_np, y_pred_np)
    
    # Sort scores and get confidence interval bounds
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    
    # Compute confidence interval
    alpha = (1.0 - ci) / 2.0
    lower_bound = sorted_scores[int(alpha * len(sorted_scores))]
    upper_bound = sorted_scores[int((1 - alpha) * len(sorted_scores))]
    
    return auprc, lower_bound, upper_bound

# Main code with minimal output for both metrics
for exp in results['DR'].keys():
    true_outcomes = []
    total_preds = []
    indices = []
    
    for i in results['DR'][exp]:
        true_outcomes.extend(i[3])
        total_preds.extend(i[4])
        indices.extend(i[5])
    
    # Create DataFrame
    df = pd.DataFrame([true_outcomes, total_preds, indices]).T
    df.columns = ['true_outcome', 'pred', 'index']
    
    try:
        # Convert tensors to numeric values
        df['true_outcome_num'] = df['true_outcome'].apply(tensor_to_numpy)
        df['pred_num'] = df['pred'].apply(tensor_to_numpy)
        df['index_num'] = df['index'].apply(tensor_to_numpy)
        
        # Group by the numeric index
        df_grouped = df.groupby('index_num')[['true_outcome_num', 'pred_num']].mean()
        
        # Calculate AUROC and bootstrap CI
        auroc, auroc_lower, auroc_upper = bootstrap_auroc_ci(
            df_grouped['true_outcome_num'], 
            df_grouped['pred_num'], 
            n_bootstraps=2000,  # Using more bootstraps for stability
            ci=0.95
        )
        
        # Calculate AUPRC and bootstrap CI
        auprc, auprc_lower, auprc_upper = bootstrap_auprc_ci(
            df_grouped['true_outcome_num'], 
            df_grouped['pred_num'], 
            n_bootstraps=10000,  # Using more bootstraps for stability
            ci=0.95
        )
        
        # Print results for both metrics
        print(f"{exp}:")
        print(f"  AUROC = {auroc:.4f}, 95% CI = [{auroc_lower:.4f}, {auroc_upper:.4f}]")
        print(f"  AUPRC = {auprc:.4f}, 95% CI = [{auprc_lower:.4f}, {auprc_upper:.4f}]")
        
    except Exception as e:
        print(f"{exp}: Error calculating metrics - {str(e)}")

In [ ]:
f = open("./results/results.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(results,f)

# close file
f.close()

In [ ]:
results = pickle.load(open('./results/results.pkl','rb'))

In [ ]:
t_stat, p_val = ttest_rel([i[0] for i in results['DR']['exp3']], [i[0] for i in results['DR']['exp6']])

print(f'The p-value is {p_val}')

